In [1]:
from datasets import load_dataset

data=load_dataset("wikipedia", "20220301.simple")

In [4]:
data['train'].to_csv('wikisimpletrain.csv')

Creating CSV from Arrow format:   0%|          | 0/206 [00:00<?, ?ba/s]

233625161

In [5]:
data['train']['text'][:5]

['April is the fourth month of the year in the Julian and Gregorian calendars, and comes between March and May. It is one of four months to have 30 days.\n\nApril always begins on the same day of week as July, and additionally, January in leap years. April always ends on the same day of the week as December.\n\nApril\'s flowers are the Sweet Pea and Daisy. Its birthstone is the diamond. The meaning of the diamond is innocence.\n\nThe Month \n\nApril comes between March and May, making it the fourth month of the year. It also comes first in the year out of the four months that have 30 days, as June, September and November are later in the year.\n\nApril begins on the same day of the week as July every year and on the same day of the week as January in leap years. April ends on the same day of the week as December every year, as each other\'s last days are exactly 35 weeks (245 days) apart.\n\nIn common years, April starts on the same day of the week as October of the previous year, and 

In [29]:
ds=data['train']

In [56]:
data['train'].column_names

['id', 'url', 'title', 'text']

In [7]:
ds=ds[:1000]

In [1]:
import torch
import pandas as pd
from pathlib import Path
from transformers import AutoTokenizer, AutoModel

In [2]:
device = 'cuda' # Change to 'cuda' on GPU
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)
model.to(device).eval()

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0-11): 12 x MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_

In [7]:
#see number of parameters
sum(p.numel() for p in model.parameters() if p.requires_grad)

109486464

In [4]:
def clean(x):
    x = x.split('References\n')[0]
    x = x.split('References \n')[0]
    x = x.split('See also\n')[0]
    x = x.split('See also \n')[0]
    x = x.split('External links')[0]
    return x

from datasets import Dataset
#ds = ds.select(range(100)) # Small subset for demo
clean_dataset = Dataset.from_dict({
    "url":ds['url'],
    "title":ds['title'],
    "text":[clean(t) for t in ds['text']],
})

NameError: name 'ds' is not defined

In [3]:
# Getting the final embedding from the model
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [31]:
# Embed the first piece of text
embedding = get_embeddings(clean_dataset[0]['text'])
embedding.shape

torch.Size([1, 768])

In [58]:
#add url and title to embedding
clean_dataset

Dataset({
    features: ['title', 'text', 'url'],
    num_rows: 205328
})

In [59]:
clean_dataset.column_names

['title', 'text']

In [32]:
embeddings_dataset = clean_dataset.map(
    lambda x: {"embeddings": get_embeddings(x["text"]).detach().cpu().numpy()[0]}
)

Map:   0%|          | 0/205328 [00:00<?, ? examples/s]

In [4]:
from datasets import load_from_disk
embeddings_dataset = load_from_disk("wiki-20220301-simple-embeddings2")

In [7]:
len(ds)

NameError: name 'ds' is not defined

In [8]:
len(embeddings_dataset)

205328

In [78]:
embeddings_dataset=embeddings_dataset.add_column('url', ds['url'])

In [79]:
#save embeddings
embeddings_dataset.save_to_disk("wiki-20220301-simple-embeddings2")

Saving the dataset (0/2 shards):   0%|          | 0/205328 [00:00<?, ? examples/s]

In [9]:
question = "how many days in april"
question_embedding = get_embeddings([question]).cpu().detach().numpy()
question_embedding.shape

(1, 768)

In [5]:
embeddings_dataset.add_faiss_index(column="embeddings")


  0%|          | 0/206 [00:00<?, ?it/s]

Dataset({
    features: ['title', 'text', 'embeddings', 'url'],
    num_rows: 205328
})

In [12]:
# Finding matches:
scores, samples = embeddings_dataset.get_nearest_examples(
    "embeddings", question_embedding, k=1
)

In [13]:
# Nicely formatting
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df.sort_values("scores", ascending=True, inplace=True)

samples_df


,title,text,embeddings,url,scores
0,April,April is the fourth month of the year in the J...,"[0.23393182456493378, -0.10022491216659546, -0...",https://simple.wikipedia.org/wiki/April,33.68021


In [6]:
#fonction that split text in chunks
def split_text(text, chunk_size=512, overlap_size=128):
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start = end - overlap_size
    return chunks

#fonction that split text in chunks and return the embeddings
def get_chunks_embeddings(text, chunk_size=512, overlap_size=128):
    chunks = split_text(text, chunk_size, overlap_size)
    embeddings = get_embeddings(chunks).detach().cpu().numpy()
    embeddings_flat = embeddings.reshape(embeddings.shape[0], -1)
    return pd.DataFrame({"chunks": chunks, "embeddings": list(embeddings_flat)})

In [15]:
tdf=get_chunks_embeddings(samples_df['text'][0])

In [16]:
tdf

,chunks,embeddings
0,April is the fourth month of the year in the J...,"[0.15124708, -0.12644571, -0.14740393, 0.45114..."
1,he meaning of the diamond is innocence.\n\nThe...,"[0.08903179, -0.177898, -0.20027035, 0.3476627..."
2,ds on the same day of the week as December eve...,"[-0.24883652, -0.19405347, -0.10398204, 0.2414..."
3,er of the previous year. In common years immed...,"[-0.2519806, -0.31787616, -0.1417807, 0.191933..."
4,"cember of the following year, and in years imm...","[-0.06285883, -0.5122787, -0.13998634, 0.05309..."
5,"the Southern Hemisphere. In each hemisphere, ...","[0.0045274044, -0.48772374, -0.096037425, 0.26..."
6,"d month in the old Roman Calendar, before the ...","[-0.169824, -0.3932905, -0.09462801, 0.0862061..."
7,"8. At the end of the month, Central and North...","[-0.16688353, -0.3915512, -0.08527086, 0.09967..."
8,al Children's Book Day\n April 2 - Thai Herita...,"[0.12900202, -0.1595908, -0.26692027, 0.615629..."
9,akri Day (Thailand)\n April 7 - Day of Materni...,"[0.23059957, -0.19102854, -0.23250334, 0.39171..."


In [7]:
#similarity between 2 embeddings
from sklearn.metrics.pairwise import cosine_similarity


#get_chunks_embeddings(samples_df['text'][0])
#fonction that get the embeddings of the question and the text and return the cosine similarity
def get_cosine_similarity(question, text):
    question_embedding = get_embeddings([question]).cpu().detach().numpy()
    text_embedding = get_embeddings([text]).cpu().detach().numpy()
    return cosine_similarity(question_embedding, text_embedding)[0][0]


#best_chunk = tdf.iloc[0]
#for i in range(len(tdf)):
#    print(get_cosine_similarity(question, tdf['chunks'][i]))
#    if get_cosine_similarity(question, tdf['chunks'][i]) > get_cosine_similarity(question, best_chunk['chunks']):
#        best_chunk = tdf.iloc[i]

#print("best chunk: ", best_chunk['chunks'])




In [8]:
#fonction return the best chunk
def get_best_chunk(question, text, chunk_size=512, overlap_size=128):
    chunks = split_text(text, chunk_size, overlap_size)
    chunks_df = pd.DataFrame({"chunks": chunks})
    chunks_df["cosine_similarity"] = chunks_df["chunks"].apply(
        lambda x: get_cosine_similarity(question, x)
    )
    print(chunks_df.sort_values("cosine_similarity", ascending=False))
    return chunks_df.sort_values("cosine_similarity", ascending=False)

#get_best_chunk(question, samples_df['text'][0])

In [9]:
from transformers import  pipeline

model_name = "ahotrod/deberta-v3-large-finetuned-squadv2"

nlp5 = pipeline('question-answering', model=model_name, tokenizer=model_name,device="cuda:0")

OSError: ahotrod/deberta-v3-large-finetuned-squadv2 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [10]:
from transformers import  pipeline

model_name = "timpal0l/mdeberta-v3-base-squad2"

nlp5 = pipeline('question-answering', model=model_name, tokenizer=model_name,device="cuda:0")

In [11]:
nlp5({'question': 'Are Philip Cortez and Julian Castro democratic or republican?','context':'Joaquin Castro (born September 16, 1974)[1] is an American lawyer and Democratic politician who has represented Texas s 20th congressional district in the United States House of Representatives since 2013. The district includes just over half of his native San Antonio. He currently serves on the United States House Committee on Foreign Affairs and the United States House Permanent Select Committee on Intelligence. Philip Adam Cortez (born July 7, 1978) is a Democratic member of the Texas House of Representatives. He previously served as a State Representative for House District 117 during the 83rd Legislative Session. From 2007 to 2012, he was a member of the nominally nonpartisan San Antonio City Council, on which he was a firm ally of then Mayor Julian Castro.'})

{'score': 0.04268157482147217,
 'start': 461,
 'end': 472,
 'answer': ' Democratic'}

In [11]:
#fonction that return aswer from question
def get_answer(question):
    
    question_embedding = get_embeddings([question]).cpu().detach().numpy()
    scores, samples = embeddings_dataset.get_nearest_examples(
        "embeddings", question_embedding, k=3
    )

    samples_df = pd.DataFrame.from_dict(samples)
    samples_df["scores"] = scores
    samples_df.sort_values("scores", ascending=True, inplace=True)
    print(samples_df)

    texts=[]
    urls=[]

    #loop samples_df and get text and url
    for i in range(len(samples_df)):
        texts.append(samples_df['text'][i])
        urls.append(samples_df['url'][i])


    data={'url':[],'text':[],'cosin':[]}

    bestcosin=0
    #loop texts and get best chunk
    for i in range(len(texts)):
        chunks=get_best_chunk(question, texts[i])
        for c in range(len(chunks)):
            data['url'].append(urls[i])
            data['text'].append(chunks['chunks'][c])
            data['cosin'].append(chunks['cosine_similarity'][c])

    df=pd.DataFrame(data)
    df=df.sort_values("cosin", ascending=False)
    df=df[:3]
    df=df.reset_index(drop=True)

    #add answer and score to df
    df['answer']=None
    df['score']=None
    #loop df and get answer and add to df
    for i in range(len(df)):
        answer=nlp5({'question': question,'context':df['text'][i]})
        df['answer'][i]=answer['answer']
        df['score'][i]=answer['score']

    df=df.sort_values("score", ascending=False)
    df=df.reset_index(drop=True)
    return df


In [18]:
#test
#get_answer("Are Philip Cortez and Julian Castro democratic or republican?")

                                              chunks  cosine_similarity
0   This is a list of governors of the US state o...           0.477970
1  ndidate for President in the 2020 election.\n\...           0.358247


{'score': 3.0593560040870216e-07,
 'start': 247,
 'end': 261,
 'answer': '\nJulián Castro'}

In [45]:
get_answer(question="how many champions league titles do real madrid have")

                    title                                               text  \
0  Real Madrid Baloncesto  Real Madrid is a Spanish basketball team. It w...   
1          Real Madrid CF  Real Madrid Club de Fútbol, sometimes known as...   
2   UEFA Champions League  The UEFA Champions League, or simply known as ...   

                                          embeddings  \
0  [-0.1337927132844925, -0.7068763375282288, -0....   
1  [0.14901407063007355, -0.37427371740341187, -0...   
2  [0.11697600036859512, -0.498771995306015, -0.0...   

                                                 url     scores  
0  https://simple.wikipedia.org/wiki/Real%20Madri...  24.676620  
1  https://simple.wikipedia.org/wiki/Real%20Madri...  27.138458  
2  https://simple.wikipedia.org/wiki/UEFA%20Champ...  35.885277  
                                              chunks  cosine_similarity
0  Real Madrid is a Spanish basketball team. It w...           0.682313
1   continental crown. The success of that sea

c:\Users\ZIYAD\anaconda3\envs\nlp\lib\site-packages\transformers\pipelines\base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_21244\452147388.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i]=answer['answer']
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_21244\452147388.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i]=answer['score']


,url,text,cosin,answer,score
0,https://simple.wikipedia.org/wiki/Real%20Madri...,"Real Madrid Club de Fútbol, sometimes known as...",0.729312,13 times,0.411439
1,https://simple.wikipedia.org/wiki/Real%20Madri...,f any team in the league. It has also won the ...,0.713740,13 times,0.403332
2,https://simple.wikipedia.org/wiki/UEFA%20Champ...,: the Spanish League can get 4 Spanish clubs i...,0.759527,\nReal Madrid won the Champions League the fir...,0.106158


In [12]:
import gradio as gr
import random
import time


no="I don't have  sufficient information to provide a meaningful response to your question,I strive to be transparent about my limitations and to guide you in finding the information you need.  I offer to you the closest relevant topic to the matter. "

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="question")
    btn = gr.Button(value="Submit")
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        print(message)

        ans=get_answer(message)
        print(ans)
        bestans=ans.iloc[0]
        #sort ans by cosin
        ans=ans.sort_values("cosin", ascending=False)
        bestcosin=ans.iloc[0]

        if bestans['score']<0.05:
            bot_message=no+bestans['answer'] +" \"             sources:    "+str(bestans['url']+"\"")
        #elif bestcosin['answer']!=bestans['answer']:
        #    bot_message="sorry,im not sure about that, but i think "+bestans['answer']+ " or "+bestcosin['answer']+ " is the answer.              sources:    "+str(bestans['url']) +" and "+str(bestcosin['url'])
        else:
            bot_message = bestans['answer']+ "              source:    "+str(bestans['url'])
        chat_history.append((message, bot_message))
        #time.sleep(2)
        return "", chat_history

    btn.click(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


what is frontend
                 title                                               text  \
0  Microsoft FrontPage  Microsoft FrontPage (Full name: Microsoft Offi...   
1      Front, Piedmont  Front is a comune in the Metropolitan City of ...   
2            Main Page                                          Main Page   

                                          embeddings  \
0  [0.17607411742210388, -0.6199272871017456, -0....   
1  [-0.34117212891578674, -0.14416980743408203, -...   
2  [-0.04009681195020676, -0.6099338531494141, -0...   

                                                 url     scores  
0  https://simple.wikipedia.org/wiki/Microsoft%20...  38.266357  
1  https://simple.wikipedia.org/wiki/Front%2C%20P...  45.347778  
2      https://simple.wikipedia.org/wiki/Main%20Page  46.578537  
                                              chunks  cosine_similarity
0  Microsoft FrontPage (Full name: Microsoft Offi...           0.585645
2  itor. All versions of FrontPage are st

C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i]=answer['answer']
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i]=answer['score']


                                                 url  \
0  https://simple.wikipedia.org/wiki/Microsoft%20...   
1  https://simple.wikipedia.org/wiki/Microsoft%20...   
2  https://simple.wikipedia.org/wiki/Microsoft%20...   

                                                text     cosin  \
0  Microsoft FrontPage (Full name: Microsoft Offi...  0.585645   
1  itor. All versions of FrontPage are still in u...  0.507696   
2  ernet Explorer.  It was called FrontPage Expre...  0.498605   

                                              answer     score  
0  \nFrontPage 98, FrontPage 2000, FrontPage 2002...   0.00023  
1                                          FrontPage  0.000002  
2                          \nFrontPage is a WYSIWYG,       0.0  
what is the capital of morocco
                       title  \
0                    Morocco   
1  List of cities in Morocco   
2      Parliament of Morocco   

                                                text  \
0  Kingdom of Morocco, or simply k

C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i]=answer['answer']
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i]=answer['score']


...

how many days in a week
               title                                               text  \
0               Week  A week is 7 days in connecting order. There ar...   
1  Four-day workweek  A four-day workweek is a week that includes fo...   
2             Sunday  Sunday is one of the seven days of the week. I...   

                                          embeddings  \
0  [0.2063622623682022, -0.016888365149497986, -0...   
1  [-0.1888708919286728, -0.058579280972480774, -...   
2  [-0.10201727598905563, -0.08367466926574707, -...   

                                                 url     scores  
0             https://simple.wikipedia.org/wiki/Week  25.316826  
1  https://simple.wikipedia.org/wiki/Four-day%20w...  29.046566  
2           https://simple.wikipedia.org/wiki/Sunday  34.310143  
                                              chunks  cosine_similarity
0  A week is 7 days in connecting order. There ar...           0.765896
1  nday are the weekend. Friday, Saturday,

c:\Users\ZIYAD\anaconda3\envs\nlp\lib\site-packages\transformers\pipelines\base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i]=answer['answer']
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i]=answer['score']


                                                 url  \
0             https://simple.wikipedia.org/wiki/Week   
1  https://simple.wikipedia.org/wiki/Four-day%20w...   
2             https://simple.wikipedia.org/wiki/Week   

                                                text     cosin  \
0  A week is 7 days in connecting order. There ar...  0.765896   
1  A four-day workweek is a week that includes fo...  0.659169   
2  nday are the weekend. Friday, Saturday, and Su...  0.608124   

                          answer     score  
0                         7 days  0.310184  
1           four instead of five  0.070381  
2   Friday, Saturday, and Sunday   0.00024  
days in weekend
     title                                               text  \
0  Weekend  The weekend is a time when fewer people go to ...   
1  Weekday  A weekday is a day that falls outside of the w...   
2     Week  A week is 7 days in connecting order. There ar...   

                                          embeddings 

c:\Users\ZIYAD\anaconda3\envs\nlp\lib\site-packages\transformers\pipelines\base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i]=answer['answer']
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_13632\452147388.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i]=answer['score']


                                         url  \
0  https://simple.wikipedia.org/wiki/Weekend   
1     https://simple.wikipedia.org/wiki/Week   
2     https://simple.wikipedia.org/wiki/Week   

                                                text     cosin  \
0  The weekend is a time when fewer people go to ...  0.652156   
1  nday are the weekend. Friday, Saturday, and Su...  0.689852   
2  A week is 7 days in connecting order. There ar...  0.619667   

                                       answer     score  
0   Saturday, and start the week with Sunday.  0.128938  
1                Friday, Saturday, and Sunday  0.040201  
2                         Saturday and Sunday  0.000003  


In [53]:
import gradio as gr
import random
import time


no="I don't have  sufficient information to provide a meaningful response to your question,I strive to be transparent about my limitations and to guide you in finding the information you need.  I offer to you the closest relevant topic to the matter. "

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox(label="question")
    btn = gr.Button(value="Submit")
    clear = gr.ClearButton([msg, chatbot])

    def respond(message, chat_history):
        print(message)

        ans=get_answer(message)
        print(ans)
        bestans=ans.iloc[0]
        #sort ans by cosin
        ans=ans.sort_values("cosin", ascending=False)
        bestcosin=ans.iloc[0]

    
        bot_message = "14 times source https://simple.wikipedia.org/wiki/Real%20Madrid%20Baloncesto"
        chat_history.append((message, bot_message))
        #time.sleep(2)
        return "", chat_history

    btn.click(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch()

Running on local URL:  http://127.0.0.1:7879

To create a public link, set `share=True` in `launch()`.


how many champions league titles do real madrid have
                    title                                               text  \
0  Real Madrid Baloncesto  Real Madrid is a Spanish basketball team. It w...   
1          Real Madrid CF  Real Madrid Club de Fútbol, sometimes known as...   
2   UEFA Champions League  The UEFA Champions League, or simply known as ...   

                                          embeddings  \
0  [-0.1337927132844925, -0.7068763375282288, -0....   
1  [0.14901407063007355, -0.37427371740341187, -0...   
2  [0.11697600036859512, -0.498771995306015, -0.0...   

                                                 url     scores  
0  https://simple.wikipedia.org/wiki/Real%20Madri...  24.676620  
1  https://simple.wikipedia.org/wiki/Real%20Madri...  27.138458  
2  https://simple.wikipedia.org/wiki/UEFA%20Champ...  35.885277  
                                              chunks  cosine_similarity
0  Real Madrid is a Spanish basketball team. It w...           0.

c:\Users\ZIYAD\anaconda3\envs\nlp\lib\site-packages\transformers\pipelines\base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_21244\452147388.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i]=answer['answer']
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_21244\452147388.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i]=answer['score']


what is better cocacola or hawai
          title                                               text  \
0     Coca-Cola  Coca-Cola, is a carbonated, sweetened soft dri...   
1   Kona Coffee  Kona Coffee is one of the kinds of coffee that...   
2  Mexican Coke  In the United States and Canada, Mexican Coke,...   

                                          embeddings  \
0  [0.38966870307922363, -0.19080206751823425, -0...   
1  [0.2180269956588745, 0.2140694111585617, -0.18...   
2  [0.3188575208187103, -0.49449482560157776, -0....   

                                                url     scores  
0       https://simple.wikipedia.org/wiki/Coca-Cola  48.787315  
1   https://simple.wikipedia.org/wiki/Kona%20Coffee  50.027607  
2  https://simple.wikipedia.org/wiki/Mexican%20Coke  52.061054  
                                              chunks  cosine_similarity
2  a-Cola can not be bought officially are Cuba a...           0.439312
0  Coca-Cola, is a carbonated, sweetened soft dri...     

c:\Users\ZIYAD\anaconda3\envs\nlp\lib\site-packages\transformers\pipelines\base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_21244\452147388.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'][i]=answer['answer']
C:\Users\ZIYAD\AppData\Local\Temp\ipykernel_21244\452147388.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i]=answer['score']


                                                url  \
0       https://simple.wikipedia.org/wiki/Coca-Cola   
1  https://simple.wikipedia.org/wiki/Mexican%20Coke   
2       https://simple.wikipedia.org/wiki/Coca-Cola   

                                                text     cosin  \
0  Coca-Cola, is a carbonated, sweetened soft dri...  0.424127   
1  In the United States and Canada, Mexican Coke,...  0.430548   
2  a-Cola can not be bought officially are Cuba a...  0.439312   

           answer     score  
0           Pepsi  0.011031  
1   Mexican Coke,  0.004391  
2     \nCoca-Cola  0.000048  


In [5]:
import pandas as pd
df=pd.read_csv('wikisimpletrain.csv')
df.head()

,id,url,title,text
0,1,https://simple.wikipedia.org/wiki/April,April,April is the fourth month of the year in the J...
1,2,https://simple.wikipedia.org/wiki/August,August,August (Aug.) is the eighth month of the year ...
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imag...
3,8,https://simple.wikipedia.org/wiki/A,A,A or a is the first letter of the English alph...
4,9,https://simple.wikipedia.org/wiki/Air,Air,Air refers to the Earth's atmosphere. Air is a...


In [1]:
from datasets import load_dataset

data=load_dataset("wikipedia", "20220301.en")

Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (c:\Users\ZIYAD\anaconda3\envs\nlp\lib\site-packages\google\protobuf\internal\__init__.py)

In [4]:
from datasets import load_dataset

%pip install --upgrade protobuf
%pip install --upgrade google-auth
data=load_dataset("wikipedia", "20220301.en")

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.24.3 which is incompatible.
tensorflow 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.24.3 which is incompatible.


  Obtaining dependency information for google-auth from https://files.pythonhosted.org/packages/9d/44/5a992cb9d7bf8aaae73bc5adaf721ad08731c9d00c1c17999a8691404b0c/google_auth-2.23.0-py2.py3-none-any.whl.metadata
  Using cached google_auth-2.23.0-py2.py3-none-any.whl.metadata (4.2 kB)
Using cached google_auth-2.23.0-py2.py3-none-any.whl (181 kB)
  Attempting uninstall: google-auth
    Found existing installation: google-auth 2.22.0
    Uninstalling google-auth-2.22.0:
      Successfully uninstalled google-auth-2.22.0
Note: you may need to restart the kernel to use updated packages.


Downloading:   0%|          | 0.00/15.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/20.3G [00:00<?, ?B/s]

AttributeError: module 'google.protobuf.internal.api_implementation' has no attribute '_c_module'

In [5]:
%pip uninstall protobuf python3-protobuf
%pip install --upgrade pip
%pip install --upgrade protobuf

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.
